In [30]:
import pandas as pd

df = pd.read_excel("../data/merged_client_loan.xlsx")

In [31]:
df.head(5)

,CLIENT_UUID,LOAN_UUID,CONTRACT_DATE,INITIAL_LOAN_AMOUNT,LOAN_END_DATE,ORIGINAL_LOAN_END_DATE,CURRENT_PRINCIPAL_OUTSTANDING,DAYS_PAST_DUE,SEGMENT,COMPANY_SIZE,INDUSTRY,SUB_INDUSTRY,COMPANY_TYPE
0,18703,7686,2023-08-30,250000.00,2024-02-29,2024-02-29,2.500000e+05,NaN,Segment 2,NaN,food & staples retailing,food distributors,single
1,23551,9212,2023-05-17,5000000.00,2024-05-17,2024-05-17,3.848048e+06,NaN,Segment 4,NaN,software & services,data processing & outsourced services,corporation
2,44879,10054,2023-03-23,456382.17,2023-09-23,2023-09-23,2.858487e+05,96.0,Segment 2,NaN,capital goods,electrical components & equipment,single
3,19436,8482,2023-07-14,3000000.00,2023-09-08,2023-09-08,7.614239e+05,NaN,Segment 3,NaN,materials,construction materials,single
4,113874,10633,2023-03-01,5000000.00,2023-09-11,2023-09-01,8.995630e+05,NaN,Segment 3,NaN,real estate,diversified reits,corporation


## Feature Engineering

In [32]:
# 1. IS_DEFAULT → loan is over 90 days past due
df['IS_DEFAULT'] = df['DAYS_PAST_DUE'] > 90

# 2. LOAN_TERM_LENGTH → duration of loan in days
df['LOAN_TERM_LENGTH'] = (df['LOAN_END_DATE'] - df['CONTRACT_DATE']).dt.days

# 3. REPEAT_BORROWER → has the client taken more than one loan?
df['REPEAT_BORROWER'] = df.duplicated(subset='CLIENT_UUID', keep=False)

# 4. COHORT_MONTH → month when each client first borrowed
first_loan_dates = df.groupby('CLIENT_UUID')['CONTRACT_DATE'].min().reset_index()
first_loan_dates.columns = ['CLIENT_UUID', 'FIRST_LOAN_DATE']
df = df.merge(first_loan_dates, on='CLIENT_UUID', how='left')
df['COHORT_MONTH'] = df['FIRST_LOAN_DATE'].dt.to_period('M').astype(str)

# Dropcolumn (not needed)
df.drop(columns=['FIRST_LOAN_DATE'], inplace=True)


In [33]:
df.head(5)

,CLIENT_UUID,LOAN_UUID,CONTRACT_DATE,INITIAL_LOAN_AMOUNT,LOAN_END_DATE,ORIGINAL_LOAN_END_DATE,CURRENT_PRINCIPAL_OUTSTANDING,DAYS_PAST_DUE,SEGMENT,COMPANY_SIZE,INDUSTRY,SUB_INDUSTRY,COMPANY_TYPE,IS_DEFAULT,LOAN_TERM_LENGTH,REPEAT_BORROWER,COHORT_MONTH
0,18703,7686,2023-08-30,250000.00,2024-02-29,2024-02-29,2.500000e+05,NaN,Segment 2,NaN,food & staples retailing,food distributors,single,False,183.0,False,2023-08
1,23551,9212,2023-05-17,5000000.00,2024-05-17,2024-05-17,3.848048e+06,NaN,Segment 4,NaN,software & services,data processing & outsourced services,corporation,False,366.0,False,2023-05
2,44879,10054,2023-03-23,456382.17,2023-09-23,2023-09-23,2.858487e+05,96.0,Segment 2,NaN,capital goods,electrical components & equipment,single,True,184.0,True,2022-08
3,19436,8482,2023-07-14,3000000.00,2023-09-08,2023-09-08,7.614239e+05,NaN,Segment 3,NaN,materials,construction materials,single,False,56.0,False,2023-07
4,113874,10633,2023-03-01,5000000.00,2023-09-11,2023-09-01,8.995630e+05,NaN,Segment 3,NaN,real estate,diversified reits,corporation,False,194.0,True,2019-03


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17997 entries, 0 to 17996
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   CLIENT_UUID                    17997 non-null  int64         
 1   LOAN_UUID                      17997 non-null  int64         
 2   CONTRACT_DATE                  17997 non-null  datetime64[ns]
 3   INITIAL_LOAN_AMOUNT            17997 non-null  float64       
 4   LOAN_END_DATE                  17994 non-null  datetime64[ns]
 5   ORIGINAL_LOAN_END_DATE         17977 non-null  datetime64[ns]
 6   CURRENT_PRINCIPAL_OUTSTANDING  17997 non-null  float64       
 7   DAYS_PAST_DUE                  2078 non-null   float64       
 8   SEGMENT                        17987 non-null  object        
 9   COMPANY_SIZE                   11075 non-null  object        
 10  INDUSTRY                       17997 non-null  object        
 11  SUB_INDUSTRY   

In [35]:
features = [
    'INITIAL_LOAN_AMOUNT',          
    'LOAN_TERM_LENGTH',            
    'REPEAT_BORROWER',             
    'INDUSTRY',                    
    'COMPANY_TYPE',                
    'SEGMENT',                                   
    'COHORT_MONTH'                 
]
target = 'IS_DEFAULT'


## Training the ML Model

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

df_model = df[features + [target]].dropna()

X = df_model[features]
y = df_model[target].astype(int) 

categorical_cols = ['INDUSTRY', 'COMPANY_TYPE', 'SEGMENT', 'COHORT_MONTH']
numeric_cols = ['INITIAL_LOAN_AMOUNT', 'LOAN_TERM_LENGTH', 'REPEAT_BORROWER']

In [37]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  
)

In [38]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  
)

In [40]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['INDUSTRY', 'COMPANY_TYPE',
                                                   'SEGMENT',
                                                   'COHORT_MONTH'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

## Evaluate

In [41]:
y_pred = pipeline.predict(X_test)

In [42]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95      3198
           1       0.70      0.45      0.55       399

    accuracy                           0.92      3597
   macro avg       0.82      0.71      0.75      3597
weighted avg       0.91      0.92      0.91      3597



In [54]:

sample_input = X_test.iloc[[1]]
true_label = y_test.iloc[1]

predicted_class = pipeline.predict(sample_input)[0]
predicted_proba = pipeline.predict_proba(sample_input)[0][1]

sample_input = sample_input.copy() 
sample_input['TRUE_LABEL'] = true_label
sample_input['PREDICTED_CLASS'] = predicted_class
sample_input['PREDICTED_PROBABILITY'] = predicted_proba
sample_input['CORRECT'] = predicted_class == true_label

print(sample_input)


       INITIAL_LOAN_AMOUNT  LOAN_TERM_LENGTH  REPEAT_BORROWER   INDUSTRY  \
17880            365498.96             643.0             True  materials   

      COMPANY_TYPE    SEGMENT COHORT_MONTH  TRUE_LABEL  PREDICTED_CLASS  \
17880  corporation  Segment 1      2019-05           1                1   

       PREDICTED_PROBABILITY  CORRECT  
17880                   0.93     True  
